<a href="https://colab.research.google.com/github/byeongdon/hanghae99/blob/main/5%EC%A3%BC%EC%B0%A8_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

시간이 없어서 일단 논문 요약으로 진행

--> 회사에서 웨비나 요약 과제가 있어 그것으로 대체 할까 고민중입니다

In [42]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [43]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [44]:
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [45]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2005.11401.pdf")
docs = loader.load()

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

In [47]:
retriever = vectorstore.as_retriever()

user_msg = f"""다음 논문을 요약해줘, 그리고 한국어로 번역해줘""" # format_docs 함수 호출
retrieved_docs = retriever.get_relevant_documents(user_msg)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 다음 논문을 요약해줘, 그리고 한국어로 번역해줘 \nContext: https://www.aclweb.org/anthology/P17-1171.\n[6] Eunsol Choi, Daniel Hewlett, Jakob Uszkoreit, Illia Polosukhin, Alexandre Lacoste, and\nJonathan Berant. Coarse-to-ﬁne question answering for long documents. In Proceedings of the\n55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers),\npages 209–220, Vancouver, Canada, July 2017. Association for Computational Linguistics. doi:\n10.18653/v1/P17-1020. URL https://www.aclweb.org/anthology/P17-1020.\n10\n\nhttps://www.aclweb.org/anthology/P17-1171.\n[6] Eunsol Choi, Daniel Hewlett, Jakob Uszkoreit, Illia Polosukhin, Alexandre Lacoste, and\nJonathan Berant. Coarse-to-ﬁne question answering f

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [48]:
response = llm.invoke(user_prompt)
print(response.content)

The paper presents a method for question answering on long documents using a coarse-to-fine approach. It first identifies relevant sections of the document and then refines the answers by analyzing those specific texts, significantly improving accuracy over traditional methods. The proposed system aims to handle the challenges posed by lengthy documents effectively.

이 논문은 긴 문서에서 질문 응답을 위한 대략적에서 세부적으로 접근하는 방법을 제시합니다. 처음에는 문서의 관련 구문을 식별하고, 이후 특정 텍스트를 분석하여 답변을 다듬어 정확성을 크게 향상시킵니다. 제안된 시스템은 긴 문서가 제기하는 문제를 효과적으로 처리하는 것을 목표로 합니다.
